In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
import numpy as np
from telewrap import Telewrap
torch.backends.cuda.matmul.allow_tf32 = True

In [2]:
model_id = "bert-base-uncased"
dataset_id = "ag_news"


In [ ]:
dataset = load_dataset(dataset_id)


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
config = AutoConfig.from_pretrained(model_id, num_labels=len(np.unique(dataset['test']['label'])))
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
training_args = TrainingArguments(
    output_dir="bert-base",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    do_train=True,
    do_eval=True,
    fp16=True, 
    gradient_accumulation_steps=8,
    logging_steps=8,
    eval_steps=8,

)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

In [9]:
with Telewrap():
    trainer.train()

Step,Training Loss
8,1.301100
16,0.961500
24,0.779800
32,0.618800
40,0.509200
48,0.441400
56,0.414600
64,0.323300
72,0.324400
80,0.282200


In [10]:
preds = trainer.predict(tokenized_datasets['test'])

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(preds.label_ids, np.argmax(preds.predictions,axis=1))

0.9394736842105263